https://www.kaggle.com/datasets/arjunbhasin2013/ccdata

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/anvar/CC GENERAL.csv')
df.dropna(inplace=True)
df.head(3)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.0,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.0,0.000000,0.00,0,12,7500.0,622.066742,627.284787,0.000000,12


In [3]:
columns = df.columns
X = df[columns[1:]]

In [4]:
X_standardized = (X - X.mean(axis=0)) / X.std(axis=0)
X_centered = X - X.mean(axis=0)

# Try KMeans with different K

In [5]:
from sklearn.cluster import KMeans
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score
import numpy as np

In [ ]:
inertia = []
results_df = pd.DataFrame()
clusters = range(2, 21)

for k in tqdm(clusters):
    
    # original
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    results_df[f'clusters_kmeans_{k}'] = kmeans.predict(X)
   
    # standardized
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X_standardized)
    results_df[f'clusters_kmeans_std{k}'] = kmeans.predict(X_standardized)
    inertia.append(kmeans.inertia_)
    
    # centered
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X_centered)
    results_df[f'clusters_kmeans_center{k}'] = kmeans.predict(X_centered)

 95%|█████████▍| 18/19 [00:18<00:01,  1.36s/it]

In [ ]:
plt.plot(clusters, inertia);
plt.xlabel('Number of clusters')
plt.ylabel('Inertia');

In [ ]:
k = 8
print(adjusted_rand_score(results_df[f'clusters_kmeans_std{k}'], results_df[f'clusters_kmeans_{k}']))

In [ ]:
k = 8
print(adjusted_rand_score(results_df[f'clusters_kmeans_std{k}'], results_df[f'clusters_kmeans_center{k}']))

In [ ]:
results_df['clusters_kmeans_std8'].value_counts().sort_index()

In [ ]:
df[(results_df['clusters_kmeans_std8'] == 0).values].mean()

In [ ]:
df[(results_df['clusters_kmeans_std8'] == 2).values].mean()

# Hierarchical clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

ac = AgglomerativeClustering(n_clusters=10,
                             affinity='euclidean',
                             linkage='single')

prediction = ac.fit_predict(X_standardized)
np.unique(prediction, return_counts=True)

In [ ]:
ac = AgglomerativeClustering(n_clusters=10,
                             affinity='euclidean',
                             linkage='complete')

prediction = ac.fit_predict(X_standardized)
np.unique(prediction, return_counts=True)


# Apply PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(2)
X_pca2 = pca.fit_transform(X_centered[columns[1:]])
print(pca.explained_variance_ratio_)

In [ ]:
k = 5
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=results_df[f'clusters_kmeans_std{k}']);

In [ ]:
components = dict(zip(columns[1:], pca.components_[0]))
components = sorted(components.items(), key=lambda x: x[1], reverse=True)

for c, w in components:
    print(c, np.round(w, 3))

# Attempt 1.

Select columns which constitutes the largest variance ratio

In [ ]:
selected_cols = ['CREDIT_LIMIT', 'PAYMENTS', 'PURCHASES', 'BALANCE', 'CASH_ADVANCE', 'ONEOFF_PURCHASES', 'MINIMUM_PAYMENTS']

In [ ]:
ac = AgglomerativeClustering(n_clusters=2,
                             affinity='euclidean',
                             linkage='single')

prediction = ac.fit_predict(X_standardized[selected_cols])
np.unique(prediction, return_counts=True)

In [ ]:
ac = AgglomerativeClustering(n_clusters=2,
                             affinity='euclidean',
                             linkage='complete')

prediction = ac.fit_predict(X_standardized[selected_cols])
np.unique(prediction, return_counts=True)

# Attempt 2. 

Use PCA representation

In [ ]:
pca = PCA(5)
X_pca5 = pca.fit_transform(X_centered[columns[1:]])
print(np.round(pca.explained_variance_ratio_.sum()*100, 2), '%')

In [ ]:
ac = AgglomerativeClustering(n_clusters=2,
                             affinity='euclidean',
                             linkage='complete')

prediction = ac.fit_predict(X_pca5)
np.unique(prediction, return_counts=True)

# Attempt 3. 

Remove outliers

In [ ]:
def detect_outliers_simple(x):
    x = x.values
    q3 = np.quantile(x, 0.75)
    q2 = np.quantile(x, 0.25)
    iqr = q3-q2
    lower_outliers = (q2-1.5*iqr) > x
    upper_outliers = (q3+1.5*iqr) < x
    return lower_outliers + upper_outliers

In [ ]:
columns = df.columns

In [ ]:
cols_outliers = dict()
outliers = [False] * df.shape[0]

for col in columns[1:]:
        cols_outliers[col] = detect_outliers_simple(df[col])
        outliers += cols_outliers[col]
        print(col, np.round(cols_outliers[col].sum() / df.shape[0] * 100), '%')

In [ ]:
X_drop_outliers = X_standardized[~outliers]

In [ ]:
X_drop_outliers.shape

only ~40% of the data left